In [24]:
import streamlit as st
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [25]:
def load_data():
    file = 'aircrahesFullDataUpdated_2024.csv'
    df = pd.read_csv(file)

    return df

In [26]:
df = load_data()

In [27]:
df.head()

,Year,Quarter,Month,Day,Country/Region,Aircraft Manufacturer,Aircraft,Location,Operator,Ground,Fatalities (air),Aboard
0,1970,Qtr 2,April,28,NaN,de Havilland Canada,de Havilland Canada DHC 6 Twin Otter 100VH,Kainatu,Airlines Australia GuineaTrans New,0,8,11
1,1952,Qtr 2,April,30,NaN,Douglas,Douglas DC 3VT,Delhi,Airways IndiaDeccan,0,9,9
2,1973,Qtr 2,April,10,NaN,Vickers Vanguard,Vickers Vanguard 952G AXOP,Near Basel,(UK) Airlines International SwitzerlandInvicta,0,108,145
3,2000,Qtr 2,April,19,'-,Antonov,Antonov AN 8TL,Pepa -,Airlines Centrafricain,0,24,24
4,1946,Qtr 2,April,30,'-,Douglas,Douglas C 47A FZ559,Pindi-Khut -,Force Air Royal,0,12,12


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5035 entries, 0 to 5034
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Year                   5035 non-null   int64 
 1   Quarter                5035 non-null   object
 2   Month                  5035 non-null   object
 3   Day                    5035 non-null   int64 
 4   Country/Region         4805 non-null   object
 5   Aircraft Manufacturer  5035 non-null   object
 6   Aircraft               5035 non-null   object
 7   Location               5035 non-null   object
 8   Operator               4801 non-null   object
 9   Ground                 5035 non-null   int64 
 10  Fatalities (air)       5035 non-null   int64 
 11  Aboard                 5035 non-null   int64 
dtypes: int64(5), object(7)
memory usage: 472.2+ KB


In [29]:
# Step 1:  Handling Missing Values and Standardizing Entries

# Replace '-' with NaN in 'Country/Region'
df['Country/Region'] = df['Country/Region'].replace('-', pd.NA)

# Fill missing values in 'Country/Region' and 'Operator' with 'Unknown'
df['Country/Region'] = df['Country/Region'].fillna('Unknown')
df['Operator'] = df['Operator'].fillna('Unknown')

In [30]:
# Step 2: Trim and Clean Text Fields
# Columns to clean: 'Country/Region', 'Aircraft', 'Location', 'Operator'
df['Country/Region'] = df['Country/Region'].str.strip().str.title()
df['Aircraft Manufacturer'] = df['Aircraft Manufacturer'].str.strip().str.title()
df['Aircraft'] = df['Aircraft'].str.strip()
df['Location'] = df['Location'].str.strip()
df['Operator'] = df['Operator'].str.strip()

In [31]:
# Step 3: Convert 'Quarter' and 'Month' to categorical data types
df['Quarter'] = pd.Categorical(df['Quarter'], categories=['Qtr 1', 'Qtr 2', 'Qtr 3', 'Qtr 4'])
df['Month'] = pd.Categorical(df['Month'], categories=[
    'January', 'February', 'March', 'April', 'May', 'June',
    'July', 'August', 'September', 'October', 'November', 'December'])

In [32]:
# Step 4: Handling Outliers
# Calculate the interquartile range (IQR) for 'Ground' and 'Fatalities (air)' to detect outliers
Q1_ground = df['Ground'].quantile(0.25)
Q3_ground = df['Ground'].quantile(0.75)
IQR_ground = Q3_ground - Q1_ground

Q1_fatalities = df['Fatalities (air)'].quantile(0.25)
Q3_fatalities = df['Fatalities (air)'].quantile(0.75)
IQR_fatalities = Q3_fatalities - Q1_fatalities

# Define outlier bounds
lower_bound_ground = Q1_ground - 1.5 * IQR_ground
upper_bound_ground = Q3_ground + 1.5 * IQR_ground

lower_bound_fatalities = Q1_fatalities - 1.5 * IQR_fatalities
upper_bound_fatalities = Q3_fatalities + 1.5 * IQR_fatalities

# Cap outliers in 'Ground' and 'Fatalities (air)'
df['Ground'] = df['Ground'].clip(lower=lower_bound_ground, upper=upper_bound_ground)
df['Fatalities (air)'] = df['Fatalities (air)'].clip(lower=lower_bound_fatalities, upper=upper_bound_fatalities)

In [33]:
# Step 5: Check for Duplicates
duplicate_rows = df.duplicated().sum()
# If any duplicates are found, remove them
df.drop_duplicates(inplace=True)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5035 entries, 0 to 5034
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Year                   5035 non-null   int64   
 1   Quarter                5035 non-null   category
 2   Month                  5035 non-null   category
 3   Day                    5035 non-null   int64   
 4   Country/Region         5035 non-null   object  
 5   Aircraft Manufacturer  5035 non-null   object  
 6   Aircraft               5035 non-null   object  
 7   Location               5035 non-null   object  
 8   Operator               5035 non-null   object  
 9   Ground                 5035 non-null   int64   
 10  Fatalities (air)       5035 non-null   float64 
 11  Aboard                 5035 non-null   int64   
dtypes: category(2), float64(1), int64(4), object(5)
memory usage: 403.9+ KB


# RESEARCH QUESTIONS

1. Which year had the highest number of fatalities (in the air)?
This question investigates the year with the most air fatalities in the dataset.

2. Which aircraft manufacturer has the highest number of accidents?
This will look at the most accident-prone manufacturers.

3. Which aircraft manufacturer has the highest total number of air fatalities?
This question focuses on identifying the aircraft manufacturer associated with the highest total number of air fatalities.

4. Which operator has been involved in the most crashes?
This question identifies the operator with the highest number of crashes.

5. What is the trend of total crashes over the years
This question provides a summary of the trend of crashes over time by showing the year with the highest number of crashes.

6. How do air fatalities vary by aircraft manufacturer?
This question investigates the total air fatalities associated with each aircraft manufacturer.

In [12]:
# Grouping data by year and summing the air fatalities
yearly_fatalities = df.groupby('Year')['Fatalities (air)'].sum().reset_index()

# Sorting to find the year with the highest fatalities
highest_fatality_year = yearly_fatalities.sort_values(by='Fatalities (air)', ascending=False).head(1)

highest_fatality_year


,Year,Fatalities (air)
61,1972,1989.5


In [35]:
# Grouping by aircraft manufacturer and counting the occurrences
manufacturer_accidents = df['Aircraft Manufacturer'].value_counts().reset_index()

# Renaming columns
manufacturer_accidents.columns = ['Aircraft Manufacturer', 'Accident Count']

# Finding the manufacturer with the highest accidents
top_manufacturer = manufacturer_accidents.head(1)

top_manufacturer


,Aircraft Manufacturer,Accident Count
0,Douglas,994


In [40]:
# Grouping by aircraft manufacturer and summing air fatalities
manufacturer_fatalities = df.groupby('Aircraft Manufacturer')['Fatalities (air)'].sum().reset_index()

# Sorting to find the manufacturer with the highest air fatalities
top_manufacturer_fatalities = manufacturer_fatalities.sort_values(by='Fatalities (air)', ascending=False).head(1)

top_manufacturer_fatalities


,Aircraft Manufacturer,Fatalities (air)
245,Douglas,16013.0


In [45]:
# Grouping by operator and counting occurrences
operator_accidents = df['Operator'].value_counts().reset_index()

# Renaming columns
operator_accidents.columns = ['Operator', 'Accident Count']

# Finding the operator with the highest accidents
top_operator = operator_accidents.head(10)

# Plotting the top 10 operators by number of accidents
plt.figure(figsize=(10, 6))
sns.barplot(x='Accident Count', y='Operator', data=top_operator)
plt.title('Top 10 Operators by Number of Accidents')
plt.xlabel('Accident Count')
plt.ylabel('Operator')
plt.show()



C:\Users\Admin\AppData\Local\Temp\ipykernel_2916\979841051.py:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [44]:
# Grouping by year and counting the number of crashes
yearly_crashes = df.groupby('Year').size().reset_index(name='Crash Count')

# Plotting the trend of crashes over the years
plt.figure(figsize=(10, 6))
sns.lineplot(x='Year', y='Crash Count', data=yearly_crashes)
plt.title('Trend of Total Crashes Over the Years')
plt.ylabel('Number of Crashes')
plt.xlabel('Year')
plt.show()



C:\Users\Admin\AppData\Local\Temp\ipykernel_2916\706807193.py:10: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [43]:
# Grouping by aircraft manufacturer and summing air fatalities
manufacturer_fatalities = df.groupby('Aircraft Manufacturer')['Fatalities (air)'].sum().reset_index()

# Sorting to see which manufacturer has the highest air fatalities
top_manufacturer_fatalities = manufacturer_fatalities.sort_values(by='Fatalities (air)', ascending=False).head(10)

# Plotting the top 10 manufacturers by air fatalities
plt.figure(figsize=(10, 6))
sns.barplot(x='Fatalities (air)', y='Aircraft Manufacturer', data=top_manufacturer_fatalities)
plt.title('Top 10 Aircraft Manufacturers by Air Fatalities')
plt.xlabel('Total Air Fatalities')
plt.ylabel('Aircraft Manufacturer')
plt.show()


C:\Users\Admin\AppData\Local\Temp\ipykernel_2916\1787652666.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
